In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120

In [2]:
import findspark
findspark.init()

In [3]:
import numpy as np

In [15]:
from source.als import AlternatingLeastSquares
from source import utils

In [16]:
spark = utils.create_spark_session()
train, test = utils.prepare_data(spark, 100000)

In [35]:
k = 5
als = AlternatingLeastSquares()

In [19]:
# als.grid_search(train, rank=[10, 20], max_iter=[10],
#                 reg_param=np.linspace(0.05, 0.3, 10), num_folds=2, metric='rmse')

als.grid_search(train, rank=[2], max_iter=[2], reg_param=[0.2], num_folds=2, metric='rmse', parallelism=4)

In [20]:
pred_ratings, pred_rankings = als.predict(test)

pd_pred_ratings = pred_ratings.toPandas()
pd_pred_rankings = pred_rankings.toPandas()

# pred_cols = ['userRanking', 'predictedRanking']
# pd_pred_rankings[pred_cols] = pd_pred_rankings.apply(
#     lambda x: pd.Series([x[pred_cols[0]][:k], x[pred_cols[1]][:k]]), axis=1)

In [21]:
als.rmse()

1.4265607873683304

In [22]:
als.precision_at_k(5)

0.9566333808844506